In [ ]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import numpy as np

In [2]:
df = pd.read_csv('arxiv.csv')
df.head()

,id,title,categories
0,704.0001,Calculation of prompt diphoton production cros...,['hep-ph']
1,704.0002,Sparsity-certifying Graph Decompositions,"['math', 'CO']"
2,704.0003,The evolution of the Earth-Moon system based o...,"['physics', 'gen-ph']"
3,704.0004,A determinant of Stirling cycle numbers counts...,"['math', 'CO']"
4,704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,"['math', 'CA']"


In [42]:
df['title'] = df['title'].apply(lambda x: x.replace("\n",""))
df['title'] = df['title'].apply(lambda x: x.strip())

shortlisted_categories = df['categories'].value_counts().reset_index(name="count").query("count > 0")["index"].tolist()
df = df[df["categories"].isin(shortlisted_categories)].reset_index(drop=True)

In [44]:
df['title'] = df['title'].str.lower()

In [51]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [52]:
df['title_tokenized'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in ('stopwords')]))

In [76]:
df["categories_tokenized"] = df['categories'].str.replace('[','')
df["categories_tokenized"] = df['categories'].str.replace(']','')
df["categories_tokenized"] = df['categories'].str.replace("'",'')

<ipython-input-76-50661faaec13>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["categories_tokenized"] = df['categories'].str.replace('[','')
<ipython-input-76-50661faaec13>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["categories_tokenized"] = df['categories'].str.replace(']','')


In [77]:
df.head()

,id,title,categories,title_tokenized,categories_tokenized
0,704.0001,calculation of prompt diphoton production cros...,['hep-ph'],calculation of prompt diphoton production cros...,[hep-ph]
1,704.0002,sparsity-certifying graph decompositions,"['math', 'CO']",sparsity-certifying graph decompositions,"[math, CO]"
2,704.0003,the evolution of the earth-moon system based o...,"['physics', 'gen-ph']",the evolution of the earth-moon system based o...,"[physics, gen-ph]"
3,704.0004,a determinant of stirling cycle numbers counts...,"['math', 'CO']",a determinant of stirling cycle numbers counts...,"[math, CO]"
4,704.0005,from dyadic $\lambda_{\alpha}$ to $\lambda_{\a...,"['math', 'CA']",from dyadic $\lambda_{\alpha}$ $\lambda_{\alpha}$,"[math, CA]"


In [78]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [79]:
docs = [TaggedDocument(doc.split(' '), [i]) for i, doc in enumerate(df.title_tokenized)]

In [80]:
model = Doc2Vec(vector_size=64, min_count=1, epochs=22)
model.build_vocab(docs)

In [81]:
model.train(docs, total_examples=model.corpus_count, epochs=model.epochs)

In [82]:
card2vec = [model.infer_vector((df['title_tokenized'][i].split(' '))) for i in range(0,len(df['title_tokenized']))]

In [84]:
vec = np.array(card2vec).tolist()
df['categories_vectors'] = vec
df.head()

,id,title,categories,title_tokenized,categories_tokenized,categories_vectors
0,704.0001,calculation of prompt diphoton production cros...,['hep-ph'],calculation of prompt diphoton production cros...,[hep-ph],"[0.004738969728350639, -0.0558941513299942, 0...."
1,704.0002,sparsity-certifying graph decompositions,"['math', 'CO']",sparsity-certifying graph decompositions,"[math, CO]","[0.12043661624193192, -0.10650545358657837, 0...."
2,704.0003,the evolution of the earth-moon system based o...,"['physics', 'gen-ph']",the evolution of the earth-moon system based o...,"[physics, gen-ph]","[-0.08601149916648865, 0.3283824026584625, 0.4..."
3,704.0004,a determinant of stirling cycle numbers counts...,"['math', 'CO']",a determinant of stirling cycle numbers counts...,"[math, CO]","[0.12976428866386414, -0.2459404319524765, 0.0..."
4,704.0005,from dyadic $\lambda_{\alpha}$ to $\lambda_{\a...,"['math', 'CA']",from dyadic $\lambda_{\alpha}$ $\lambda_{\alpha}$,"[math, CA]","[0.07599613815546036, -0.04482531175017357, 0...."


In [85]:
df.to_csv('arxiv_embeddings.csv', index=False) 